# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [42]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [43]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [44]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = sorted(set(text))
    #vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    #int_to_vocab = {word: ii for ii, word in enumerate(vocab, 1)}
    #vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [45]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {'.': '||Period||',
            ',': '||Comma||',
            '"': '||Quotation_Mark||',
            ';': '||Semicolon||',
            '!': '||Exclamation_Mark||',
            '?': '||Question_mark||',
            '(': '||Left_Parentheses||',
            ')': '||Right_Parentheses||',
            '\n': '||Return||',
            '--': '||Dash||'}
    #return None

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1


C:\Users\mchiappe\AppData\Local\Continuum\anaconda3\envs\dog-project\lib\site-packages\ipykernel\__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [49]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    #Input = tf.placeholder(tf.int32, [batch_size, num_steps], name='Input')
    #Targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='Targets')
    #LearningRate = tf.placeholder(tf.int32, [batch_size, num_steps], name='LearningRate')
    #Input = tf.placeholder(tf.int32, shape=[None,None], name='Input')
    #Targets = tf.placeholder(tf.int32, shape=[None,None], name='Targets')
    #LearningRate = tf.placeholder(tf.float32, name='LearningRate')
    input = tf.placeholder(tf.int32, (None,None), 'input')
    Targets = tf.placeholder(tf.int32, (None,None), 'Targets')
    LearningRate = tf.placeholder(tf.float32, None, 'LearningRate')
    return input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [50]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * 3)
    # Getting an initial state of all zeros
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name="initial_state")
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [51]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [52]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    Outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state,'final_state')
    return Outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [53]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    input_data = get_embed(input_data,vocab_size,rnn_size)
    outputs,final_state = build_rnn(cell,input_data)
    logits = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn = None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [54]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    batches = []
    
    # number of batches
    n_batches = len(int_text)//(batch_size * seq_length)
    
    characters_per_batch = batch_size * seq_length
    
    # required number of words
    required_number_words = (n_batches * batch_size * seq_length)
    
    # clip arrays to ensure we have complete batches for inputs, targets same but moved one unit over
    input_data = np.array(int_text[ : (n_batches * characters_per_batch)])
    target_data = np.array(int_text[1 : (n_batches * characters_per_batch)+1])
    
    target_data[-1] = input_data[0]
    
    inputs = input_data.reshape(batch_size, -1)
    #inputs = input_data.reshape(n_batches, -1)
    targets = target_data.reshape(batch_size, -1)
    #targets = target_data.reshape(n_batches, -1)
    
    #inputs = np.split(inputs.reshape(batch_size*(len(int_text)/batch_size)),n_batches,1)
    #inputs = np.split(inputs, n_batches,1)
    #targets = np.split(targets, n_batches,1)
    inputs = np.split(inputs, n_batches, -1)
    targets = np.split(targets, n_batches, -1)
    
    
    batches = np.array(list(zip(inputs, targets)))
    #batches = batches.reshape(num_batches, 2, batch_size, seq_length)
    
    return batches
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [72]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 100
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [74]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.821
Epoch   0 Batch    1/21   train_loss = 8.577
Epoch   0 Batch    2/21   train_loss = 6.896
Epoch   0 Batch    3/21   train_loss = 6.390
Epoch   0 Batch    4/21   train_loss = 6.280
Epoch   0 Batch    5/21   train_loss = 6.567
Epoch   0 Batch    6/21   train_loss = 6.685
Epoch   0 Batch    7/21   train_loss = 6.900
Epoch   0 Batch    8/21   train_loss = 6.708
Epoch   0 Batch    9/21   train_loss = 6.654
Epoch   0 Batch   10/21   train_loss = 6.762
Epoch   0 Batch   11/21   train_loss = 6.663
Epoch   0 Batch   12/21   train_loss = 6.736
Epoch   0 Batch   13/21   train_loss = 7.092
Epoch   0 Batch   14/21   train_loss = 6.888
Epoch   0 Batch   15/21   train_loss = 6.758
Epoch   0 Batch   16/21   train_loss = 7.000
Epoch   0 Batch   17/21   train_loss = 6.743
Epoch   0 Batch   18/21   train_loss = 6.796
Epoch   0 Batch   19/21   train_loss = 6.899
Epoch   0 Batch   20/21   train_loss = 6.897
Epoch   1 Batch    0/21   train_loss = 6.713
Epoch   1 

Epoch   8 Batch   15/21   train_loss = 6.228
Epoch   8 Batch   16/21   train_loss = 6.337
Epoch   8 Batch   17/21   train_loss = 6.153
Epoch   8 Batch   18/21   train_loss = 6.200
Epoch   8 Batch   19/21   train_loss = 6.266
Epoch   8 Batch   20/21   train_loss = 6.273
Epoch   9 Batch    0/21   train_loss = 6.241
Epoch   9 Batch    1/21   train_loss = 6.291
Epoch   9 Batch    2/21   train_loss = 6.218
Epoch   9 Batch    3/21   train_loss = 6.229
Epoch   9 Batch    4/21   train_loss = 6.148
Epoch   9 Batch    5/21   train_loss = 6.207
Epoch   9 Batch    6/21   train_loss = 6.153
Epoch   9 Batch    7/21   train_loss = 6.286
Epoch   9 Batch    8/21   train_loss = 6.210
Epoch   9 Batch    9/21   train_loss = 6.177
Epoch   9 Batch   10/21   train_loss = 6.198
Epoch   9 Batch   11/21   train_loss = 6.139
Epoch   9 Batch   12/21   train_loss = 6.145
Epoch   9 Batch   13/21   train_loss = 6.384
Epoch   9 Batch   14/21   train_loss = 6.243
Epoch   9 Batch   15/21   train_loss = 6.265
Epoch   9 

Epoch  17 Batch    9/21   train_loss = 6.127
Epoch  17 Batch   10/21   train_loss = 6.142
Epoch  17 Batch   11/21   train_loss = 6.075
Epoch  17 Batch   12/21   train_loss = 6.070
Epoch  17 Batch   13/21   train_loss = 6.301
Epoch  17 Batch   14/21   train_loss = 6.155
Epoch  17 Batch   15/21   train_loss = 6.186
Epoch  17 Batch   16/21   train_loss = 6.300
Epoch  17 Batch   17/21   train_loss = 6.104
Epoch  17 Batch   18/21   train_loss = 6.148
Epoch  17 Batch   19/21   train_loss = 6.193
Epoch  17 Batch   20/21   train_loss = 6.213
Epoch  18 Batch    0/21   train_loss = 6.185
Epoch  18 Batch    1/21   train_loss = 6.226
Epoch  18 Batch    2/21   train_loss = 6.146
Epoch  18 Batch    3/21   train_loss = 6.174
Epoch  18 Batch    4/21   train_loss = 6.091
Epoch  18 Batch    5/21   train_loss = 6.137
Epoch  18 Batch    6/21   train_loss = 6.082
Epoch  18 Batch    7/21   train_loss = 6.218
Epoch  18 Batch    8/21   train_loss = 6.139
Epoch  18 Batch    9/21   train_loss = 6.107
Epoch  18 

Epoch  26 Batch    3/21   train_loss = 6.037
Epoch  26 Batch    4/21   train_loss = 5.938
Epoch  26 Batch    5/21   train_loss = 5.956
Epoch  26 Batch    6/21   train_loss = 5.920
Epoch  26 Batch    7/21   train_loss = 6.077
Epoch  26 Batch    8/21   train_loss = 5.995
Epoch  26 Batch    9/21   train_loss = 5.926
Epoch  26 Batch   10/21   train_loss = 5.996
Epoch  26 Batch   11/21   train_loss = 5.975
Epoch  26 Batch   12/21   train_loss = 5.950
Epoch  26 Batch   13/21   train_loss = 6.137
Epoch  26 Batch   14/21   train_loss = 5.986
Epoch  26 Batch   15/21   train_loss = 6.037
Epoch  26 Batch   16/21   train_loss = 6.129
Epoch  26 Batch   17/21   train_loss = 5.939
Epoch  26 Batch   18/21   train_loss = 5.947
Epoch  26 Batch   19/21   train_loss = 5.951
Epoch  26 Batch   20/21   train_loss = 5.954
Epoch  27 Batch    0/21   train_loss = 5.961
Epoch  27 Batch    1/21   train_loss = 5.986
Epoch  27 Batch    2/21   train_loss = 5.918
Epoch  27 Batch    3/21   train_loss = 5.981
Epoch  27 

Epoch  34 Batch   18/21   train_loss = 4.832
Epoch  34 Batch   19/21   train_loss = 4.854
Epoch  34 Batch   20/21   train_loss = 4.832
Epoch  35 Batch    0/21   train_loss = 4.881
Epoch  35 Batch    1/21   train_loss = 4.880
Epoch  35 Batch    2/21   train_loss = 4.748
Epoch  35 Batch    3/21   train_loss = 4.879
Epoch  35 Batch    4/21   train_loss = 4.810
Epoch  35 Batch    5/21   train_loss = 4.786
Epoch  35 Batch    6/21   train_loss = 4.774
Epoch  35 Batch    7/21   train_loss = 4.914
Epoch  35 Batch    8/21   train_loss = 4.869
Epoch  35 Batch    9/21   train_loss = 4.855
Epoch  35 Batch   10/21   train_loss = 4.854
Epoch  35 Batch   11/21   train_loss = 4.769
Epoch  35 Batch   12/21   train_loss = 4.723
Epoch  35 Batch   13/21   train_loss = 4.888
Epoch  35 Batch   14/21   train_loss = 4.588
Epoch  35 Batch   15/21   train_loss = 4.745
Epoch  35 Batch   16/21   train_loss = 4.884
Epoch  35 Batch   17/21   train_loss = 4.780
Epoch  35 Batch   18/21   train_loss = 4.756
Epoch  35 

...

Epoch 156 Batch   18/21   train_loss = 0.345
Epoch 156 Batch   19/21   train_loss = 0.323
Epoch 156 Batch   20/21   train_loss = 0.341
Epoch 157 Batch    0/21   train_loss = 0.423
Epoch 157 Batch    1/21   train_loss = 0.348
Epoch 157 Batch    2/21   train_loss = 0.382
Epoch 157 Batch    3/21   train_loss = 0.377
Epoch 157 Batch    4/21   train_loss = 0.391
Epoch 157 Batch    5/21   train_loss = 0.344
Epoch 157 Batch    6/21   train_loss = 0.385
Epoch 157 Batch    7/21   train_loss = 0.363
Epoch 157 Batch    8/21   train_loss = 0.468
Epoch 157 Batch    9/21   train_loss = 0.380
Epoch 157 Batch   10/21   train_loss = 0.411
Epoch 157 Batch   11/21   train_loss = 0.396
Epoch 157 Batch   12/21   train_loss = 0.377
Epoch 157 Batch   13/21   train_loss = 0.344
Epoch 157 Batch   14/21   train_loss = 0.325
Epoch 157 Batch   15/21   train_loss = 0.341
Epoch 157 Batch   16/21   train_loss = 0.313
Epoch 157 Batch   17/21   train_loss = 0.342
Epoch 157 Batch   18/21   train_loss = 0.322
Epoch 157 

Epoch 165 Batch   12/21   train_loss = 0.291
Epoch 165 Batch   13/21   train_loss = 0.269
Epoch 165 Batch   14/21   train_loss = 0.261
Epoch 165 Batch   15/21   train_loss = 0.269
Epoch 165 Batch   16/21   train_loss = 0.250
Epoch 165 Batch   17/21   train_loss = 0.253
Epoch 165 Batch   18/21   train_loss = 0.238
Epoch 165 Batch   19/21   train_loss = 0.210
Epoch 165 Batch   20/21   train_loss = 0.227
Epoch 166 Batch    0/21   train_loss = 0.302
Epoch 166 Batch    1/21   train_loss = 0.249
Epoch 166 Batch    2/21   train_loss = 0.290
Epoch 166 Batch    3/21   train_loss = 0.283
Epoch 166 Batch    4/21   train_loss = 0.303
Epoch 166 Batch    5/21   train_loss = 0.261
Epoch 166 Batch    6/21   train_loss = 0.299
Epoch 166 Batch    7/21   train_loss = 0.260
Epoch 166 Batch    8/21   train_loss = 0.327
Epoch 166 Batch    9/21   train_loss = 0.260
Epoch 166 Batch   10/21   train_loss = 0.289
Epoch 166 Batch   11/21   train_loss = 0.276
Epoch 166 Batch   12/21   train_loss = 0.276
Epoch 166 

Epoch 174 Batch    6/21   train_loss = 0.315
Epoch 174 Batch    7/21   train_loss = 0.295
Epoch 174 Batch    8/21   train_loss = 0.359
Epoch 174 Batch    9/21   train_loss = 0.308
Epoch 174 Batch   10/21   train_loss = 0.339
Epoch 174 Batch   11/21   train_loss = 0.319
Epoch 174 Batch   12/21   train_loss = 0.306
Epoch 174 Batch   13/21   train_loss = 0.273
Epoch 174 Batch   14/21   train_loss = 0.266
Epoch 174 Batch   15/21   train_loss = 0.282
Epoch 174 Batch   16/21   train_loss = 0.256
Epoch 174 Batch   17/21   train_loss = 0.287
Epoch 174 Batch   18/21   train_loss = 0.274
Epoch 174 Batch   19/21   train_loss = 0.252
Epoch 174 Batch   20/21   train_loss = 0.273
Epoch 175 Batch    0/21   train_loss = 0.368
Epoch 175 Batch    1/21   train_loss = 0.287
Epoch 175 Batch    2/21   train_loss = 0.314
Epoch 175 Batch    3/21   train_loss = 0.296
Epoch 175 Batch    4/21   train_loss = 0.313
Epoch 175 Batch    5/21   train_loss = 0.256
Epoch 175 Batch    6/21   train_loss = 0.311
Epoch 175 

Epoch 183 Batch    0/21   train_loss = 0.583
Epoch 183 Batch    1/21   train_loss = 0.501
Epoch 183 Batch    2/21   train_loss = 0.518
Epoch 183 Batch    3/21   train_loss = 0.550
Epoch 183 Batch    4/21   train_loss = 0.584
Epoch 183 Batch    5/21   train_loss = 0.553
Epoch 183 Batch    6/21   train_loss = 0.595
Epoch 183 Batch    7/21   train_loss = 0.567
Epoch 183 Batch    8/21   train_loss = 0.650
Epoch 183 Batch    9/21   train_loss = 0.612
Epoch 183 Batch   10/21   train_loss = 0.643
Epoch 183 Batch   11/21   train_loss = 0.624
Epoch 183 Batch   12/21   train_loss = 0.577
Epoch 183 Batch   13/21   train_loss = 0.571
Epoch 183 Batch   14/21   train_loss = 0.584
Epoch 183 Batch   15/21   train_loss = 0.607
Epoch 183 Batch   16/21   train_loss = 0.570
Epoch 183 Batch   17/21   train_loss = 0.638
Epoch 183 Batch   18/21   train_loss = 0.622
Epoch 183 Batch   19/21   train_loss = 0.610
Epoch 183 Batch   20/21   train_loss = 0.626
Epoch 184 Batch    0/21   train_loss = 0.705
Epoch 184 

Epoch 191 Batch   15/21   train_loss = 1.768
Epoch 191 Batch   16/21   train_loss = 1.620
Epoch 191 Batch   17/21   train_loss = 1.711
Epoch 191 Batch   18/21   train_loss = 1.638
Epoch 191 Batch   19/21   train_loss = 1.661
Epoch 191 Batch   20/21   train_loss = 1.575
Epoch 192 Batch    0/21   train_loss = 1.656
Epoch 192 Batch    1/21   train_loss = 1.600
Epoch 192 Batch    2/21   train_loss = 1.673
Epoch 192 Batch    3/21   train_loss = 1.659
Epoch 192 Batch    4/21   train_loss = 1.613
Epoch 192 Batch    5/21   train_loss = 1.594
Epoch 192 Batch    6/21   train_loss = 1.558
Epoch 192 Batch    7/21   train_loss = 1.580
Epoch 192 Batch    8/21   train_loss = 1.596
Epoch 192 Batch    9/21   train_loss = 1.595
Epoch 192 Batch   10/21   train_loss = 1.726
Epoch 192 Batch   11/21   train_loss = 1.657
Epoch 192 Batch   12/21   train_loss = 1.611
Epoch 192 Batch   13/21   train_loss = 1.525
Epoch 192 Batch   14/21   train_loss = 1.482
Epoch 192 Batch   15/21   train_loss = 1.581
Epoch 192 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [75]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [76]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [77]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [78]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    next_word = np.random.choice(list(int_to_vocab.values()),1,p=probabilities)[0]
    return next_word

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [79]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: who'da thought such that hope, you were aging big dollars.
homer_simpson: yeah, it'll were jack working in in complete in tonight since were war.
patty_bouvier: will believe the woman, moe! but a throw, get the best thing is a mouth soon that could do it--...(then) hi, moe. no of the bees, we were all put on the beach of day. is you get me out?
homer_simpson:(reading)" the big deal?
homer_simpson:(gasps) it it is out!
homer_simpson:(holding up) if i go to the home in a window you.
homer_simpson: i spy, this nonsense.
homer_simpson:(to bears) been here for the pipe open.
homer_simpson: i could be homer? go ahead!
homer_simpson:(grabs annoyed desperate) my fresco. homer the thing would us been their every friends, drinkin' after where and he gets, and i mean you? and" if you know, you can't call these corner, right!


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.